In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
1
import pandas as pd
import librosa as la
import librosa.display as ld
from IPython.display import Audio
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
%reload_ext watermark
%watermark -a "Zero Zero" --iversions

Author: Zero Zero

numpy     : 1.23.5
matplotlib: 3.7.1
librosa   : 0.10.0.post2
seaborn   : 0.12.2
pandas    : 2.0.0
sklearn   : 1.2.2



In [ ]:
paths = [os.path.join(name) for name in os.listdir("./data")]
paths_bee = []
paths_no_bee = []
for path in paths:
if "No_Bee" in path:
paths_no_bee.append(path)
else:
paths_bee.append(path)
len(paths_no_bee), len(paths_bee)

In [ ]:
rnd = np.random.randint(0, len(paths_bee))
data, sr = la.load(f"./data/{paths_bee[rnd]}", sr=44100, res_type="kaiser_best")
print("Canais ", len(data.shape))
print("# total de amostras ", data.shape[0])
print("Arquivo: ", paths_bee[rnd])
print("Taxa de amostragem ", sr)
print("Duração ", la.get_duration(y=data, sr=sr))
Audio(data=data, rate=sr)
# plt.title("SOM ABELHA", size=16)
# ld.waveshow(y=data, sr=sr)
# plt.show();

In [ ]:
# rnd = np.random.randint(0, len(paths_bee))
# data, sr = la.load(f"./data/{paths_bee[rnd]}", sr=44100, res_type="kaiser_best")
# stft = la.stft(data)
# stft_db = la.amplitude_to_db(np.abs(stft))
# ld.specshow(stft_db, x_axis="time", y_axis="log", cmap="Spectral")
# plt.show();

In [ ]:
# rnd = np.random.randint(0, len(paths_bee))
# data, sr = la.load(f"./data/{paths_bee[rnd]}", sr=44100, res_type="kaiser_best")
# mfccs = la.feature.mfcc(y=data, sr=sr, n_mfcc=40)
# mfccs_db = la.amplitude_to_db(np.abs(mfccs))
# ld.specshow(mfccs_db, x_axis="time", y_axis="log", cmap="Spectral")
# plt.show()

In [ ]:
rnd = np.random.randint(0, len(paths_no_bee))
data, sr = la.load(f"./data/{paths_no_bee[rnd]}", sr=44100, res_type="kaiser_best")

print("Canais ", len(data.shape))
print("# total de amostras ", data.shape[0])
print("Arquivo: ", paths_no_bee[rnd])
print("Taxa de amostragem ", sr)
print("Duração ", la.get_duration(y=data, sr=sr))
Audio(data=data, rate=sr)
# plt.title("SOM SEM ABELHA", size=16)
# ld.waveshow(y=data, sr=sr)
# plt.show()

In [ ]:
# rnd = np.random.randint(0, len(paths_no_bee))
# data, sr = la.load(f"./data/{paths_no_bee[rnd]}", sr=44100,␣
↪res_type="kaiser_best")
# stft = la.stft(data)
# stft_db = la.amplitude_to_db(np.abs(stft))
# plt.show(ld.specshow(stft_db, x_axis="time", y_axis="log", cmap="Spectral"));

In [ ]:
# rnd = np.random.randint(0, len(paths_no_bee))
# data, sr = la.load(f"./data/{paths_no_bee[rnd]}", sr=44100,␣
↪res_type="kaiser_best")
# mfccs = la.feature.mfcc(y=data, sr=sr, n_mfcc=40)
# mfccs_db = la.amplitude_to_db(np.abs(mfccs))
# ld.specshow(mfccs_db, x_axis="time", y_axis="log", cmap="Spectral")
# plt.show()

In [ ]:
def features_extrator(file_name):
    data, sample_rate = la.load(f"./data/{file_name}", sr=44100, res_type="kaiser_best")
    mfccs_features = la.feature.mfcc(y=data, sr= sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    return mfccs_features_scaled

extracted_features_bee = []
extracted_features_nobee = []

for path in tqdm(paths_bee):
    try:
        data = features_extrator(path)
        extracted_features_bee.append(data)
    except:
        print(path)
        continue
    
for path in tqdm(paths_no_bee):
    try:
        data = features_extrator(path)
        extracted_features_nobee.append(data)
    except:
        print(path)
        continue

In [ ]:
len(extracted_features_bee), len(extracted_features_nobee)

In [ ]:
df = pd.DataFrame({"feature": extracted_features_bee, "class": 1})
df2 = pd.DataFrame({"feature": extracted_features_nobee, "class": 0})
data = pd.concat([df, df2])

In [ ]:
X = np.array(data["feature"].tolist())
y = np.array(data["class"].tolist())

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)

In [ ]:
clf_lr = LogisticRegression()
clf_lr.fit(X_train, y_train)
predict = clf_lr.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)
predict = clf_nb.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)
predict = clf_knn.predict(X_test)

print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
predict = clf_rf.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
clf_svm = SVC()
clf_svm.fit(X_train, y_train)
predict = clf_svm.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
estimators = [("lr", LogisticRegression()),
("svm", SVC()),
("rf", RandomForestClassifier())]

In [ ]:
clf_st = StackingClassifier(estimators=estimators)
clf_st.fit(X_train, y_train)
predict = clf_st.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)

In [ ]:
clf_vt = VotingClassifier(estimators=estimators)
clf_vt.fit(X_train, y_train)
predict = clf_vt.predict(X_test)
print(accuracy_score(y_test, predict))
print(classification_report(y_test, predict))
confusion_matrix(y_test, predict)